In [1]:
from model.BPRMF import *
from preprocess.AmazonBook import *
from evaluation.BPRMF_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/amazon-book'
dataset = AmazonBook(path)

data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 1e-4,
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
}
model = BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device)

# SISA Method

In [2]:
config['epochs'] = 120
config['num_shards'] = 5
model_list =[]

for _ in range(config['num_shards']):
    model_list.append(BPRMF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    embedding_dim = config['embedding_dim']
).to(device))

shard_models, shards, epoch_tracks, test_topks = sisa_BPRMF_eva(model_list, config, data, device)

Epoch: 001, Loss: 0.6949, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 002, Loss: 0.6938, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 003, Loss: 0.6927, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 004, Loss: 0.6916, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 005, Loss: 0.6905, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 006, Loss: 0.6895, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 007, Loss: 0.6884, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 008, Loss: 0.6873, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 009, Loss: 0.6863, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 010, Loss: 0.6852, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 011, Loss: 0.6841, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 012, Loss: 0.6831, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 013, Loss: 0.6821, HR@20: 0.0001, Recall@20: 0.0002, NDCG@20: 0.0002
Epoch: 014, 

# Split the dataset

In [3]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [4]:
shard_models = sisa_BPRMF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
sisa_BPRMF_forget_data_eva(shard_models, forget_data, config, device)